<b>Short list of content.</b>
1. Look on data.
    1. Data with multiple possibility to answer at once.
        1. Describe data. Look at data,transform it.
        2. Trying to find correlations. Visualization.
    2. Data with only one possible answer.
        1. Transform. Correlate. Gender conspiracy theory xD.
        2. Visualization.
2. A little bit more visualization.

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from collections import Counter
import warnings
%matplotlib inline

In [ ]:
warnings.filterwarnings('ignore')
data=pd.read_csv('../input/kaggle-survey-2020/kaggle_survey_2020_responses.csv')

<b>Intro</b>

Before starting analizing and plotting, I've decided to make some functions and other auxiliary things<br>
to make easier this process. To make shorter code.<br>
The first version of function was fuction which needed only question's number.<br>
<b>Give question's number to func and it made for you bar plot<br></b>
But later, I've changed my mind,and made func which returned data for bar plot and have more input params,<br>
it makes more variations of usage.<br>
I though about making this func more complicated,variable but had not time because of personal issue.<br>
Sorry for that and lack of beauty.<br>
Just take a look! :)

In [ ]:
#Auxiliary data which helps to show visualisation.
#Type [question position(offset) in DF,amount of columns cointained in the question in DF]
k=1
kk=0
sss=[]
for i in range(1,len(data.columns)):
    try:
        if (data.columns[i][:3]==data.columns[i+1][:3]):          
            k+=1           
        else:
            sss.append([kk+1,k])
            kk+=k
            k=1
    except IndexError:
        sss.append([kk+1,k])
        kk+=k
        k=1

In [ ]:
# This function takes question's number and data as input and generate data for visualisation
def vis_func(x,xdata,sort=False):
    temp_data=xdata.iloc[1:,sss[x-1][0]:sum(sss[x-1])]
    if sss[x-1][1]!=1: 
        data_count=temp_data.count()
        col_ar=[]
        for i in temp_data:
            col_ar.append(temp_data[i].value_counts().keys()[0])
        
        return [col_ar, data_count.values]
    
    else:
        col2_ar=[]
        for i in temp_data.value_counts(sort=sort).keys():
            col2_ar.append(i[0])
        
        return [col2_ar,temp_data.value_counts(sort=sort)]
    

#First version    

def vis_func_1st(x,xdata,sort=False):
    temp_data=xdata.iloc[1:,sss[x-1][0]:sum(sss[x-1])]
    
    if sss[x-1][1]!=1: 
        data_count=temp_data.count()
        col_ar=[]
        for i in temp_data:
            col_ar.append(temp_data[i].value_counts().keys()[0])
        
        fig=plt.figure(figsize=(16,8))
        plt1=fig.add_subplot()
        plt1.set_xticklabels(col_ar, rotation=45)
        plt1.bar(col_ar, height=data_count.values,label=xdata.iloc[0,sss[x-1][0]])
        plt1.legend()
        
    else:
        col2_ar=[]
        for i in temp_data.value_counts(sort=sort).keys():
            col2_ar.append(i[0])
        
        fig=plt.figure(figsize=(16,8))
        plt1=fig.add_subplot()
        plt1.set_xticklabels(col2_ar, rotation=45)
        plt1.bar(col2_ar, height=temp_data.value_counts(sort=sort),label=xdata.iloc[0,sss[x-1][0]])
        plt1.legend()

At the beginning,I decided to look at the distribution of programming languages and the distribution of recommended languages.<br>
We can see,that Python is most popular language and most recommended.
Percent of recommended to using other languages by users,which using it, quite low.

In [ ]:
fig=plt.figure(figsize=(16,8))
plt1=fig.add_subplot()
plt1.set_xticklabels(vis_func(7,data)[0], rotation=45)

plt1.bar(vis_func(7,data)[0], height=vis_func(7,data)[1],label='Using')
plt1.bar(vis_func(8,data)[0], height=vis_func(8,data)[1],label='Recommended')
plt1.legend()
plt.show()

One things, which I interested, after I saw this python's domination,was ,<b> how many people recommend an aspiring data scientist Python, dependence of</b> are they using Python or if they aren't using Python.

In [ ]:
print('% of people using Python and recommended it:', \
round((len(data[data['Q7_Part_1']==data['Q8']].index)/data['Q7_Part_1'].count())*100,2))

In [ ]:
print('% of people using other languages,but recommending Python',\
round(len(data[(data['Q7_Part_1'].isnull()==True)&(data['Q8']=='Python')].index)\
/len(data[(data['Q7_Part_1'].isnull()==True)].index)*100,2))

As we see, not all Python's users recommending Python,those people about 1/6.<br>
But more that 1/4 of people using other languages recommending Python.<br>

Let's look to languages distribution scaled by respondent's age.<br>
As we can see on the plot, in respondet's age groups older 35y.o. percent python's usage fluctuate between 35-50,<br>
whereas in respondents younger age, python's usage fluctuate between 75-80 percent.<br>
So, Python going more and more popular.


In [ ]:
data22=data[(data['Q7_Part_1'].isnull()==False)] #sorted by Python
data23=data[(data['Q7_Part_1'].isnull()==True)]  #other data 
fig=plt.figure(figsize=(16,8))
plt1=fig.add_subplot()

plt1.set_xticklabels(vis_func(1,data)[0], rotation=45)
plt1.text(3, 3500,'Programming lagnuage users by age',  fontsize=15, fontweight='bold')
plt1.bar(vis_func(1,data22)[0], height=vis_func(1,data22)[1],label='Python users')
plt1.bar(vis_func(1,data23)[0], height=vis_func(1,data23)[1],label='Other')
plt1.legend()
plt.show()

After it,I decided to split data for data where we can chose only one option and data with many options at once.<br>
The reason of this decision was, that I wanted to see, how many options people choose. What's mean and median value. How often people chose a maximum of option's possible number.<br>
Is there any correlation in how many options respondents chose from question to question?<br>

In [ ]:
Columns=data.columns
columns_s=[] # list of answers with only one possible answer

for i in Columns:
    if (('Part' not in i) and ('OTHER' not in i)):
        columns_s.append(i)
        
columns_s=columns_s[1:] # drop Time from Start to Finish
#print(columns_s)
data_s=data[columns_s] # data with only one possible answer
data_m=data.drop(columns_s+['Time from Start to Finish (seconds)'],axis=1) # data with multiple possibility to answer
#data_s
#data_m

In [ ]:
data_m_1=data_m.notna().drop(index=0)
data_m_1[data_m_1==True]=1 # data where value=1 if option choosen, esle 0.
#data_m_1

In [ ]:
data_m_1['ZEROS']=0
d=data_m_1['ZEROS'] # temporary columns with zero value
columns_m_1=data_m_1.columns
data_for_corr_m_1=pd.DataFrame() #merged data,which contain amounts of chosen options in question 

for i in range(len(columns_m_1)):
    try:
        if (('Part' in columns_m_1[i]) or ('OTHER' in columns_m_1[i])) and (columns_m_1[i][:3]==columns_m_1[i+1][:3]):
            d+=data_m_1.iloc[:,i]          
        else:
            d+=data_m_1.iloc[:,i]
            data_for_corr_m_1[columns_m_1[i][:5]]=d
            d=0
    except IndexError:
        d+=data_m_1.iloc[:,i]
        data_for_corr_m_1[columns_m_1[i][:5]]=d
        d=0
    
data_for_corr_m_1=data_for_corr_m_1.drop('ZEROS',axis=1)

Just short describe of merged data,which contain amounts of chosen options in question below.

In [ ]:
data_for_corr_m_1.iloc[:,:10].describe()

In [ ]:
data_for_corr_m_1.iloc[:,10:20].describe()

In [ ]:
data_for_corr_m_1.iloc[:,20:].describe()

As we can see, we have non-popular question, which more than half respondents skip<br>
Also we have maximum of option's number chosen in each question.<br>
Below, I've checked it and saw, that not a lot of anomaly rows.<br>
So, I've decided to dont del it from frame.</br>

In [ ]:

index2=[]
for i in data_for_corr_m_1:
    index2.append(data_for_corr_m_1[data_for_corr_m_1[i]>=data_for_corr_m_1[i].max()].index)

index2_t=[]
index2_2t=[]
index2_n=[]
for i in index2:
    index2_2t.append(len(i))
    for j in i:
        index2_n.append(j)
        if j not in index2_t:
            index2_t.append(j)

row_to_del=[] #list of index's number, with a lot of max options per respondent
for i in Counter(index2_n):
    if Counter(index2_n)[i]>5:
        row_to_del.append(i)
print('Number of respondents,which one chose maximum options in more than 5 question')
print(len(row_to_del))

In [ ]:
print('Mean amount of chosen options value in all questions ')
print(np.mean(data_for_corr_m_1.mean()))
print('Median amount of chosen options value in all questions ')
print(np.mean(data_for_corr_m_1.median()))

In [ ]:
columns_m_1_pop=[] # list of popular questions 
dict_pop=(data_for_corr_m_1.describe().loc[['50%']]>0).any()
for i in dict_pop.keys():
    if dict_pop[i]==True:
        columns_m_1_pop.append(i)
        
print('Mean amount of chosen options value in most popular questions')
print(np.mean(data_for_corr_m_1[columns_m_1_pop].mean()))
print('Median amount of chosen options value in most popular questions')
print(np.mean(data_for_corr_m_1[columns_m_1_pop].median()))

In [ ]:
sns.heatmap(data_for_corr_m_1.corr())
plt.show()

So. As we can see, we haven't correlation between questions block 'A' and block 'B' it's ok,<br>
because of survey methodic.<br>
But we have strong correlation inside this blocks, it's could mean, that people chose a proportional<br>
number of answers options inside questions block. They generally chose a lot or not so.<br>
Also we can see, that we have more correlation between questions,which close to each other.<br>
Respondents more often chose the same percent of options amount in neighbor questions.<br>
And we have not bad correlations in first part of questions. Let's look on it too.<br>
For this we should remove Block A,B and 'non popular' questions.

In [ ]:
data_for_corr_m_1_cut=data_for_corr_m_1[columns_m_1_pop]
data_for_corr_m_1_cut.columns=(['Programming languages',"IDE's",'Notebook host','Hardware','Vizualization Lib',
                                'ML Frameworks','ML Algorithm','Work activities','Courses','Media'])
data_for_corr_m_1_cut.corr()                             

Now we can take a look closer to this part<br>
We have good correlation between amount of chosen programming languages and IDE's, that's clear.<br>
So, respondents, who use a lot of visualization libraries, also use a lot of<br>
ML Frame works and many different ML algorithms, and vice versa. That's also clear :)<br>
Also interesting thing,that people,who using a lot of online education platforms, also<br>
are active consumers of online sources,that report on data science topics.And vice versa.<br>

In [ ]:
sns.heatmap(data_for_corr_m_1_cut.corr())
plt.show()

Sorry,if it's not so interesting, but i had have interest to confirm some things by data.<br>
By the next step, we will check of what compensation depend or not depend.

In this part, I've try to check dependencies between respondent's age,Education,<br>
programming and ML expirience,compensation and gender.<br>
For this I've created dictionary. Keys are values of original data, and new values:<br>
For all exept education and gender it is mean of category value.<br>
Education its categorial data, but it can be scaled. All known,that <br>
high school degree<<Doctoral degree and etc.<br>
And I've replace in gender data man->2,woman->1.<br>
<b>Note! I'm don't think,that womans<man and etc. its just for corr!</b><br>
It was deliberate. I've try to check out next thing, is there any dependencies,<br>
between gender and salary(compensation). If yes and mans earn more money,<br>
then we will have good positive correlation(for example. 2(man)>1(woman)<br>
100k dollars > 70k dollars),else we'll have negative or no correlation.


In [ ]:
#auxiliary dict for data replacement
dict_conv={'35-39':37.0, '30-34': 32.0, '22-24': 23.0, '25-29':27.0, '18-21':19.0,
 '55-59':57.0, '50-54':52.0, '40-44':42.0, '60-69':65.0, '45-49':47.0, '70+':70.0,'Doctoral degree':5.0,
 'Master’s degree':4.0, 'Bachelor’s degree':3.0, 'No formal education past high school':1.0,
 'Some college/university study without earning a bachelor’s degree':2.0,
 'Professional degree':6.0, 'I prefer not to answer':0.0,'5-10 years':8.0,'10-20 years':15.0, '3-5 years':4.0,'< 1 years':0.5,
 '1-2 years':1.5, '20+ years':20.0, 'I have never written code':0.0, 'I do not use machine learning methods':0.0,
 '3-4 years':3.5, 'Under 1 year':0.5, '2-3 years':2.5, '4-5 years':4.5, '20 or more years':20.0, '100,000-124,999':112500.0, 
 '15,000-19,999':17500.0, '125,000-149,999':137500.0, '70,000-79,999':75000.0, '30,000-39,999':35000.0, '90,000-99,999':95000.0,
 '1,000-1,999':1500.0, '$0-999':500.0, '10,000-14,999':12500.0, '150,000-199,999':175000.0, '60,000-69,999':65000.0, 
 '4,000-4,999':4500.0, '> $500,000':500000.0, '300,000-500,000':400000.0, '40,000-49,999':45000.0, '25,000-29,999':27500.0, 
 '80,000-89,999':85000.0, '7,500-9,999':8750.0, '50,000-59,999':55000.0, '250,000-299,999':275000.0, '5,000-7,499':6250.0, 
 '2,000-2,999':2500.0, '20,000-24,999':22500.0, '200,000-249,999':225000.0, '3,000-3,999':3500.0, 'Man':2.0, 'Woman':1.0,
 'Prefer to self-describe':0.0, 'Prefer not to say':0.0, 'Nonbinary':0.0}

In [ ]:
#replacing data
Columns_s_tr=['Q1','Q4','Q6','Q15','Q24','Q2']
data_s_c=data[Columns_s_tr].fillna(0)

for i in dict_conv:
    data_s_c[data_s_c==i]=dict_conv[i]

data_s_c.columns=['Age','Education','Programming Exp','ML exp','Compensation','Gender']
data_s_c_2= data_s_c.drop(np.where((data_s_c['Compensation']==0)|(data_s_c['Compensation']==500))[0])

#data_s_c_2

In [ ]:
data_s_c.iloc[1:].astype(float).corr()

So. What we can see?
We see,that <b>Age</b> has strong correlation with <b>Programming expirience</b>.<br>
Good correlation with <b>Compensation and ML expirience</b>, and not bad<br>
with <b>Education</b>.<br>
Also we can see,that here <b>NO correlation</b> between <b>gender</b> and all other data.<br>
<b>Compensation</b> has good correlation with <b>programming and ML expirience</b>.<br>
We have dependancy between <b>ML expirience and Programming expirience.</b>

In [ ]:
sns.heatmap(data_s_c.iloc[1:].astype(float).corr())
plt.show()

Further, I've decided to find more dependencies. And add more data for it.<br>
And found good correlation between <b>Compensation</b> and <br>
how many activities that make up an important part of your role at work,<br>
respondents are chose.<br>
Also we have good dependency between <b>ML expirience</b> and how many options was<br>
chosen in question about <b>ML Frameworks,Algorithm and Vizualisation lib</b><br>

In [ ]:
data_merg=data_s_c.merge(data_for_corr_m_1_cut,how='outer',left_index=True,right_index=True)
sns.heatmap(data_merg.iloc[1:].astype(float).corr())
plt.show()

Further,we will look to more plots! :)

We can see,that more wealthiest group is in range 30-50 y.o.

In [ ]:
data_t_1=data.iloc[1:].drop(np.where(data['Q24']=='$0-999')[0]).sort_values(by=['Q24'])
data_t_1_1=pd.crosstab(data_t_1['Q1'],data_t_1['Q24'])
plt.figure(figsize=(16,8))
ax=sns.heatmap(data_t_1_1,linewidth=0.01)
ax.set(xlabel='Compensation', ylabel='Age')
ax.text(7, -0.5,'Compensation by age without $0-900',  fontsize=15, fontweight='bold')
plt.show()

Interesting thing,that almost no woman with programming expirience more,than 10 years.<br>
And almost no woman older 50 y.o.

In [ ]:
data_t_2=pd.crosstab(data['Q2'].iloc[1:],data['Q6'].iloc[1:])
plt.figure(figsize=(16,8))
ax=sns.heatmap(data_t_2,linewidth=0.01)
ax.set(xlabel='Programming Exp', ylabel='Gender')
ax.text(2,-0.5,'Gender by Programming expirience',  fontsize=15, fontweight='bold')
plt.show()

In [ ]:
data_t_3=pd.crosstab(data['Q2'].iloc[1:],data['Q1'].iloc[1:])
plt.figure(figsize=(16,8))
ax=sns.heatmap(data_t_3,linewidth=0.01)
ax.set(xlabel='Age', ylabel='Gender')
ax.text(4,-0.5,'Gender by Age',  fontsize=15, fontweight='bold')
plt.show()

As we saw, we had no correlation between gender and compensation.<br>
And here we can see it too. We have womans in the same categories of compensation,<br>
as mans.But no a lot of them. If we look to 2nd plot, we will see about 10 womans and 40 mans,<br>
with compensation >500.000$. If we scale it by 3rd plot(respondents gender distribution),<br>
we will see,that's same value of percent.<br>
It's explained by respondent's gender distribution.

In [ ]:
data_t_4=data.iloc[1:].drop(np.where(data['Q24']=='$0-999')[0])
data_t_4_1=pd.crosstab(data_t_4['Q2'],data_t_4['Q24'])
plt.figure(figsize=(16,8))
ax=sns.heatmap(data_t_4_1,linewidth=0.01)
ax.set(xlabel='Compensation', ylabel='Gender')
ax.text(6,-.5,'Compensation without $0-900 by gender',  fontsize=15, fontweight='bold')
plt.show()

In [ ]:
data_t_4=data[data['Q24']=='> $500,000'].iloc[1:]
data_t_4_1=pd.crosstab(data_t_4['Q2'],data_t_4['Q24'])
plt.figure(figsize=(12,8))
ax=sns.heatmap(data_t_4_1)
ax.set(xlabel='Compensation', ylabel='Gender')
ax.text(0,-.5,'Compensation > $500,000 by gender',  fontsize=15, fontweight='bold')
plt.show()

In [ ]:
vis_func_1st(2,data)

Top 20 countries

In [ ]:
fig=plt.figure(figsize=(16,8))
plt1=fig.add_subplot()
plt1.set_xticklabels(vis_func(3,data,1)[0], rotation=45)

plt1.bar(vis_func(3,data,1)[0][:20], height=vis_func(3,data,1)[1][:20],label="Respondent's countries, Top 20")
plt1.legend()
plt.show()

<b>Let's do just quick look through all data.

In [ ]:
for i in range(1,len(sss)+1):
    if i in [1,2,3,7,8]:
        continue
    else:
        vis_func_1st(i,data)